# Deploy ML DNN Model Notebook

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Data-Reading" data-toc-modified-id="Data-Reading-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Data Reading</a></span></li><li><span><a href="#Model-Loading" data-toc-modified-id="Model-Loading-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Model Loading</a></span></li></ul></li></ul></div>

## Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
np.set_printoptions(suppress=True)

## Modeling

### Data Reading

In this project, example data is included to test the model. The data is provided as csv file and can be read using pandas:

In [2]:
raw_df = pd.read_csv('input/example_data.csv', index_col=0)
raw_df

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Europe,Japan,USA
9,8,390.0,190.0,3850.0,8.5,70,0,0,1
25,8,360.0,215.0,4615.0,14.0,70,0,0,1
28,8,304.0,193.0,4732.0,18.5,70,0,0,1
31,4,113.0,95.0,2228.0,14.0,71,0,1,0
33,6,232.0,100.0,2634.0,13.0,71,0,0,1
...,...,...,...,...,...,...,...,...,...
369,4,112.0,88.0,2395.0,18.0,82,0,0,1
375,4,105.0,74.0,1980.0,15.3,82,1,0,0
382,4,108.0,70.0,2245.0,16.9,82,0,1,0
384,4,91.0,67.0,1965.0,15.7,82,0,1,0


- Notes: we need to use `index_col=0` since this first column is not a feature but an index.

### Model Loading

The model is also included with the project files under [dnn_model](dnn_model) folder. This model can be loaded with keras:

In [3]:
path = "dnn_model"
model = keras.models.load_model(path) # or tf.saved_model.load(path)

__Model Structure__:

In [4]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 9)                 19        
_________________________________________________________________
dense_5 (Dense)              (None, 64)                640       
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 4,884
Trainable params: 4,865
Non-trainable params: 19
_________________________________________________________________


- Notes: 
    - Normalization step is within the model itself. Therefore, data preprocessing is needed for this model.
    - The model takes 9 features which are: `['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Europe', 'Japan', 'USA']`.
    - After loading, the model is ready to receive an input and provide the output which is a prediction for a car `Auto MPG`.

__Prediction__:

Our API is designed to receive a single instance of features to provide a prediction. However, the model expects a numpy array of inputs. Therefore, we need to wrap a single input inside another numpy array using one of the following ways:

In [5]:
model.predict(np.expand_dims(raw_df.iloc[0], axis=0))

array([[15.818567]], dtype=float32)

In [6]:
model.predict(raw_df[:1].values)

array([[15.818567]], dtype=float32)

In [7]:
model.predict(np.array(list(raw_df.iloc[0]),ndmin=2, dtype=np.float32))

array([[15.818567]], dtype=float32)

In [8]:
model.predict(np.array([8.0, 390.0, 190.0, 3850.0, 8.5, 70.0, 0.0, 0.0, 1.0], ndmin=2)).squeeze().tolist()

15.818567276000977

In [9]:
model.predict(raw_df[:10])

array([[15.818567],
       [10.656727],
       [11.357668],
       [25.113348],
       [20.905853],
       [12.893879],
       [13.36429 ],
       [12.654336],
       [19.55999 ],
       [31.92345 ]], dtype=float32)